(chap:astrodriz)=
# Creating Mosaics with `AstroDrizzle`

While not strictly necessary, you may find it useful to generate a mosaic of all of the *HST* observations for a galaxy, creating a single image of the full galaxy from the combined individual observations.
For example, the galaxy M81 has over 27 unique *HST* fields, not all of which have enough X-ray sources within them that could be used for an astrometric correction (see {ref}`chap:astrometry`). For that reason, I found it useful to combine all of the fields into a single mosaic, so that I could perform an astrometric correction on the entire image using what few reference objects there were. *Even if you don't need a full mosaic*, you should still try to align all of your fields to one another with `TweakReg`, as this will reduce headaches down the line. 

Creating a mosaic is usually a very time-consuming process that requires a lot of trial and error. The best resource is the official `DrizzlePac` notebook[^1]: 
https://spacetelescope.github.io/hst_notebooks/notebooks/DrizzlePac/align_mosaics/align_mosaics.html

Because this process takes up a lot of memory, it is best you run these steps through the command line rather than an `iPython` notebook, which will more than likely time out and crash. If you use command line `python`, you'll want to monitor its memory usage with the Activity Monitor app to ensure no single step is taking too many resources and causing the kernel to crash.

If running `Drizzlepac` through the command line is still causing too many issues with your machine, then you may need to run it out of **SciServer**, a site that provides free storage and computing resources for heavy-duty code: https://apps.sciserver.org/

Keep in mind, you will have to reinstall the packages you need (such as `drizzlepac`) within SciServer and will probably need to run the code through a SciServer command line terminal to keep it from crashing. See {ref}`chap:starting` for the list of packages you will need. It might be a good idea to set up `stenv` on SciServer as well.

(sec:astrodriz-files)=
## Downloading HST Observations with `astroquery`

Regardless of whether you decided to manually find *HST* images or via `pyVO`, in order to align and combine your images, you will need to download the appropriate calibration files. `Drizzlepac` requires `FLC` files, which are charge transfer efficiency (CTE) corrected. I've tested this code with `DRZ` files (which are pre-drizzled), but unfortunately I could not get those to align and mosaic, so for now, `FLC` files are the way to go.

To find the appropriate files, query MAST/HST with `astroquery`: 

```
from astroquery.mast import Observations

print("Finding observations for ACS/WFC images...")
obs_table = Observations.query_criteria(objectname="M101", \
                  obstype="all", obs_collection="HST", \
                  filters=["F435W", "F555W", "F814W"], \
                  proposal_id=[9490], instrument_name="ACS/WFC")
print("Done")
```

You don't strictly need to know the proposal ID for `astroquery` to find *HST* images, but it helps to narrow down the download to the correct images. Here, I used the information I gathered when selecting the images I wanted with `pyVO` (which is saved under the header `ObsID`).

After querying the observations, download the resulting datafiles. Don't be alarmed if it tells you it's downloading thousands of files; in actuality, it will only download as many files as is listed in `obs_table` above. *However*, if the program seems to get stuck on a particular download (meaning it takes more than 5 minutes for a single file), you may need to rerun it to make sure it hasn't glitched.

```
print("Downloading data.\n This may take several minutes... \n")
products = Observations.get_product_list(obs_table)
data_prod = ['FLC']       # other options are ['FLT','DRC','DRZ']
data_type = ['HAP-SVM']   # other options are ['CALACS','CALWF3','CALWP2']

Observations.download_products(products, project=data_type, \
                               productSubGroupDescription=data_prod)
print("Done")
```

Finally, clean up the downloaded directories by pulling the appropriate files and moving them to the main working directory:

```
import shutil
import glob
import os

# Gathering the FLC file names
input_flcs = glob.glob(os.path.join('mastDownload','HST','*','*.fits'))

# Moving files from mastDownload directory into current directory
for flc in input_flcs:
  shutil.copy(flc, os.path.basename(flc))

shutil.rmtree('mastDownload') # removes the extra directories
print("Done")
```

Occasionally you'll encounter an empty FITS file, which will raise an error during the drizzling step:
```
ValueError: Input file '<filename>' is neither a GEIS file nor a FITS file
```

(see {ref}`chap:errors`). As far as I can tell, this may happen if the file isn't downloaded properly. You can tell which files are bad by attempting to open them:

```
from astropy.io import fits

flc_files = glob.glob("hst*_acs_wfc_*_j*_flc.fits")
flc_files.sort()       # Alphabetizing files

for i,f in enumerate(flc_files):
    try: hdu = fits.open(f)
    except: print(f,"not good. Please remove.")
```

You can then delete any bad files (such as with `os.remove(f)`) and redownload them either through the `astroquery` or manually through the MAST/HST website[^2]. When selecting files through this portal, you'll want to uncheck the `Minimum Recommended Products` box and check the `FLC` box under `Groups` in `My Downloads Basket`. Beware: if you choose to skip bad files rather than redownload them, you may end up with some messy images, as drizzling observations will stack them to create a cleaner image sans cosmic rays. 

(sec:tweakreg)=
## Aligning HST Fields with `TweakReg`

When aligning *HST* fields, I prefer to align each filter separately. I don't actually know if this is a required step, but I feel it allows for better control and prevents too much memory to be used up at once. In order to ensure each filter is aligned to one another, you should define a single reference file, which is controlled with the `refimage` parameter. In this example, I choose to use the first alphabetical `FITS` file for the F555W filter, but there may be various reasons to be more discerning with your file choice. In general, it's a good idea to use a field that has very good observations as your baseline, so doing a quick visual inspection of your fields before starting this process never hurts. 

One more thing to note: It's a good idea for your first run to have `expand_refcat` and `writecat` both set to `True`, if your computational resources will allow. This will create a catalog of stars against which the remaining fields can be matched, making it more likely for your mosaic to come out successfully. The reference catalog will be saved as `cumulative_sky_refcat_*.coo` and can be called with the `refcat` parameter (be sure to set `expand_refcat=False` and `writecat=False` for the remaining `TweakReg` runs). You may need to increase the `threshold` parameter to save on computational resources while generating the reference catalog file, but this can be adjusted in subsequent runs.

```
from drizzlepac import tweakreg

# Starting with F555W, collecting all F555W FLC files
flc_files = glob.glob("hst*_acs_wfc_f555w_j*_flc.fits")
flc_files.sort()       # Alphabetizing files
reffile = flc_files[0] # Selecting a reference file to use on all FITS

# An example of the first run of TweakReg
# The next run will set expand_refcat=False, writecat=False
# and call the refcat file created by this code

tweakreg.TweakReg(flc_files, refimage=reffile, \
                  imagefindcfg={'threshold':100, 'conv_width':3.5}, \
                  expand_refcat=True, writecat=True, \
                  enforce_user_order=False, \
                  fitgeometry='rscale', \ # allows shift, rotation & rescale
                  minobj=3, sigma=3.0, nclip=3, \
                  searchrad=1.0, searchunits='arcseconds', \
                  peakmax=70000, ylimit=0.3, interactive=False, \
                  shiftfile=True, reusename=True, \
                  outshifts='M101_shifts_acs_f555w.txt', \
                  updatehdr=False)
```

The first time you run `TweakReg`, you'll want to set `savehdr` to `False` until you're sure you like the alignment. The parameters you'll likely want to change if things don't go as expected are: 

* `threshold`: Decrease to search for fainter objects. However, this increases the computational load of the code, which may result in crashes. If you find your terminal killing your `python` run, increasing this number is the first thing you should try.
* `searchrad`: Increasing the search radius can help find matches, but this may increase the computational time and result in false matches. (Default is 1)
* `minobj`: Decrease minimum number of objects to match when running the fit if not enough matches are found for a field. (Default is 15)
* `sigma`: Decrease the source clipping limit if not enough matches are identified. (Default is 3)
* `nclip`: Number of clipping iterations to run. Decreasing this can decrease the runtime. (Default is 3) 

You should check the shift solutions by pulling some statistics from the shift file:

```
from astropy.table import Table

shift_table = Table.read('M101_shifts_acs_f555w.txt', \
                         format='ascii.no_header', \
                         names=['file', 'dx', 'dy', 'rot', \
                                'scale', 'xrms', 'yrms'])
                                
formats = ['.2f', '.2f', '.3f', '.5f', '.2f', '.2f']
for i, col in enumerate(shift_table.colnames[1:]):
    shift_table[col].format = formats[i]
```

```{figure} tweakreg_stats.png
:name: fig-tweakreg-stats
:width: 500px

Example statistics from a run of `TweakReg`. In this case, the shifts are greater than the RMS, but the rotation angles are very small (i.e. likely not significant), so `TweakReg` should be rerun with `fitgeometry='shift'`. Ideally, none of rows will contain `NaN` values, as this suggests the fitting for that field failed. This may be resolved by using a reference star catalog. Note: the first field is the reference field, so no shifts are needed.
```

{numref}`fig-tweakreg-stats` is an example of the output from the code above (but from a F435W run). If all values are small, then the fields are already well-aligned and the header does not need to be updated. If `dx` and `dy` are smaller than the RMS, but `rot` is significant, then the fields only need a rotation to be well-aligned. If `rot` is small, as is the case in {numref}`fig-tweakreg-stats`, then it is recommended[^3] to rerun `TweakReg` with `fitgeometry='shift'`. Allowing `TweakReg` to do a rotational shift when one is not needed can result in distortions in the field images, causing the final product to look blurry in places (see {numref}`fig-tweakreg-rscale` below), so it's crucial you choose the correct fitting geometry for your needs! 


```{figure} tweakreg_rscale.png
:name: fig-tweakreg-rscale

Example of a mosaic between 2 *HST* fields created using `fitgeometry='rscale'` when no rotation is needed  (see {numref}`fig-tweakreg-stats`). Notice how the field on the right is clear, but the field on the left is blurry and misaligned. The blur effect is caused by distortions from rotation.
```

`TweakReg` will also save a series of residual images, which you can use to check the individual shifts (see {numref}`fig-tweakreg-resids` below). Ideally, the residuals will cluster around zero. A better test, however, is to pull the `FITS` file up in your image processing software (`DS9`) and see how they align, which you can do after you apply the alignment as described in {ref}`sec:astrodrizzle`.

```{figure} tweakreg_resids.png
:name: fig-tweakreg-resids
:width: 500px

Example of residuals of a good alignment made by `TweakReg`.
```

If you like the alignment, then you can rerun `drizzlepac` with `savehdr=True`. This will save the shifts in the coordinate system to the header of the `FITS` files. Check your shifts visually with an image processing application. In `DS9`, this can be done by opening an RGB frame (`Frame > New RGB Frame`) and opening overlapping fields under each of the 3 filters (make sure you set the scale to `zscale` in order to see them properly). Check that the stars in their overlapping regions align. Also check if the filters for a single field align with one another, creating a single, coherent color image. If you're happy with the alignments, move on to the next section. If not, you can try running different parameters through `drizzlepac` until you are. 

(sec:astrodrizzle)=
## Creating Mosaics with `AstroDrizzle`

One you are satisfied with your alignment, you can stitch together each field into a single mosaic using `AstroDrizzle`:

```
from drizzlepac.astrodrizzle import AstroDrizzle as adriz

adriz(flc_files, output='M101_mosaic_acs_f555w', \
      preserve=False, clean=True, context=False, \
      skymethod='match', \ # renormalizes the brightness of each field
      driz_sep_bits='4096', driz_cr_corr=True, \
      final_bits='4096', final_refimage=reffile, \
      final_wcs=True, final_rot=0, \  # places North pointed up
      configobj=None, overwrite=True, build=False)

# Setting build=False will create a separate FITS file with just the 
# science data under PRIMARY. If build=True, the science data will be 
# saved as a separate extension in the FITS file labeled SCI.
```

Check the resulting `FITS` file (here, `M101_mosaic_acs_f555w.fits`) in `DS9`. If everything looks good, repeat these steps for each of the other filters, being sure to use the same reference file for all alignments. Using the same reference file should allow `AstroDrizzle` to align all filters to the same coordinates.

Once all mosaics are complete, check the full color mosaic in `DS9` as an RGB frame, playing close attention to how the alignment of each field looks.

```{figure} M101_mosaic.png
:name: fig-M101-mosaic
:width: 750px

Full color mosaic of M101 produced by `AstroDrizzle`.
```

```{note}
You may need to adjust the scale parameters of each filter to make the colors look natural against each other, due to the different intensities and exposure times of each filter. This is done by selecting `Scale > Scale Parameters` and adjusting the maximum scale (the minimum can usually be set to zero). Typically, the red, green, and blue filters have something close to a 3:2:1 maximum scale ratio. Red will almost always have the highest maximum scale parameter, while blue will almost always have the lowest. In {numref}`fig-M101-mosaic`, my maximum RGB scale parameters are 0.5, 0.2, and 0.15, respectively. 
```

[^1]: Or you can read the DrizzlePac Handbook: https://www.stsci.edu/files/live/sites/www/files/home/scientific-community/software/drizzlepac/_documents/drizzlepac-handbook-v1.pdf
[^2]: https://mast.stsci.edu/portal/Mashup/Clients/Mast/Portal.html
[^3]: https://hst-docs.stsci.edu/drizzpac/chapter-7-data-quality-checks-and-trouble-shooting-problems/7-2-verifying-tweakreg-solutions-after-user-reprocessing